<a href="https://colab.research.google.com/github/itsdawei/david-lee-twitter/blob/main/CAIS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/CAIS++/Project/

DATASET_DIR = './dataset/data.csv'
EMBEDDINGS_DIR = './glove.6B.50d.txt'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/CAIS++/Project


In [22]:
from sklearn.utils import shuffle
import pandas as pd


data = pd.read_csv(DATASET_DIR)
valences = data['valence'].values
authors = data['author'].values
tweets = data['tweet'].values

PERCENT_DATA = .05
tweets_shuffled, labels_shuffled = shuffle(tweets, valences)

total = int(len(tweets_shuffled) * PERCENT_DATA)

tweets_shuffled = tweets_shuffled[:total]
labels_shuffled = labels_shuffled[:total]

In [14]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

EMBEDDING_DIM = 50

def load_valence_data(tweets, valences, embeddings_dir):
	# Load tweets, labels
  print("1 -- Loading tweets and labels")
  labels = valences//4


	# Tokenize the tweets (convert sentence to sequence of words)
  print("2 -- Tokenizing the tweets: converting sentences to sequence of words")
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(tweets)
  sequences = tokenizer.texts_to_sequences(tweets)
  word_index = tokenizer.word_index

	# Pad sequences to ensure samples are the same size
  print("3 -- Padding sequences to ensure samples are the same size")
  training_data = pad_sequences(sequences)
  
  print("4 -- Loading pre-trained word embeddings. This may take a few minutes.")
  embeddings_index = {}
  f = open(embeddings_dir,'rb')
  for line in f:
      values = line.split()
      word = values[0].decode('UTF-8')
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()

  print("5 -- Finding word embeddings for words in our tweets.")
  # prepare word embedding matrix
  num_words = len(word_index)+1
  embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
  for word, i in word_index.items():
      if i >= num_words:
          continue
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

  return tweets, training_data, labels, word_index, embedding_matrix


In [15]:
tweets, tweets_preprocessed, labels, word_index, embedding_matrix = load_valence_data(tweets_shuffled, labels_shuffled, EMBEDDINGS_DIR)

1 -- Loading tweets and labels
2 -- Tokenizing the tweets: converting sentences to sequence of words
3 -- Padding sequences to ensure samples are the same size
4 -- Loading pre-trained word embeddings. This may take a few minutes.
5 -- Finding word embeddings for words in our tweets.


In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tweets_preprocessed, labels, train_size=0.8, test_size=0.2, shuffle=True, random_state=0)

In [35]:
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dropout, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras import metrics
from keras.models import Model

model = Sequential()

# Add pre-trained embedding layer 
# converts word indices to GloVe word embedding vectors as they're fed in
model.add(Embedding(len(word_index) + 1,
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=tweets_preprocessed.shape[1],
                    trainable=False))

# First LSTM layer
model.add(LSTM(64, return_sequences = True, activation='relu'))
model.add(Dropout(.2))

# Second LSTM layer 
model.add(LSTM(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

print(model.summary())

LOSS = 'categorical_crossentropy'
OPTIMIZER = 'RMSprop'

model.compile(loss = LOSS, optimizer = OPTIMIZER, metrics = [metrics.binary_accuracy])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 39, 50)            6943300   
                                                                 
 lstm_18 (LSTM)              (None, 39, 64)            29440     
                                                                 
 dropout_17 (Dropout)        (None, 39, 64)            0         
                                                                 
 lstm_19 (LSTM)              (None, 64)                33024     
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 7,005,829
Trainable params: 62,529
Non-trainable params: 6,943,300
_________________________________________________________________
None


In [ ]:
TEST_SIZE = 0.5

EPOCHS = 10
BATCH_SIZE = 128

model.fit(x_train, y_train, 
          epochs = EPOCHS, 
          batch_size = BATCH_SIZE, 
          validation_split = TEST_SIZE)

Epoch 1/10


KeyboardInterrupt: ignored

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 
